In [1]:
import os
import numpy as np
import pandas as pd

Code for adding two new sites based on random atom placement

Inputs : 

    -A lattice_output.txt File present in this directory (same directory as this code) 
    
    -additional_sites names = ['fccPd','hcpPd']
    
    -no_added_atoms intended to be in the lattice
    
    -added_site_type_number = [3,4] (needs to be different from that already used in the lattice_output file provided)
    
    
Output: An explicit lattice structure input file lattice_input.dat ready to use for kMC

### Inputs:

In [2]:
Input_File_name = 'lattice_output.txt'
no_added_atoms = 20
additional_sites = ['fccPd','hcpPd']
added_site_type_number = [3,4]

Output_File_name = 'lattice_input.dat'

#-----

In [3]:
#Reading from lattice output file
file=open(Input_File_name,'r').readlines()
b=[]
for i in np.arange(len(file)): 
    b.append(file[i].split())                   #Dividing the rows into columns
o = pd.DataFrame(data=b)                        #Final output
#Removing first few rows with 0 in first column 
o = o[o.iloc[:,0] != '0' ]
#Removing any empty lines
o = o.dropna()

#Converting non-coordinate columns from string numbers to int for better referencing
o[o.columns[0]] = o.iloc[:,0].astype(int)
o[o.columns[3:]] = o.iloc[:,3:].astype(int)

#Extracting total number of sites
Tot_n_sites = o.iloc[-1,0]

#Extracting Max coordinate
max_cord = o.iloc[:,4].max()

#Extracting default site types
def_site_types = o.iloc[:,3].unique()

#Extracting default number of site types
def_no_site_types = len(def_site_types)

# #site names as manual inputs
site_names = ['fcc','hcp']

In [4]:
def sequence_hex(def_df,n_add,hcp,fcc):
    def_df = o
    n_add = no_added_atoms
    hcp = added_site_type_number[1]
    fcc = added_site_type_number[0]

    #Check if the added site type numbers already exist in dataframe
    if def_df.iloc[:,3].isin([hcp,fcc]).any():
        raise Exception("Error: The site type numbers are taken")

    #Calculating matrix of replacement

    new_df = def_df
    Tot_n_sites = def_df.iloc[-1,0]

    n_size = int(np.sqrt(Tot_n_sites/2)) #the i size of ixi square lattice
    mat_size = (n_size**2) * 2 #Matrix size
    matrix = np.arange(2, mat_size+1, 2).reshape(n_size, n_size) #Creating an even number lattice matrix for diamond sequence
    matrix

    matrix = np.delete(matrix, (-1), axis=1) #Delete last column (Which translates to removing the top layer as candidate for starting the new orders)
    matrix = np.delete(matrix, (-1), axis=0) #Delete last row (Which translates to removing the furthest layer as candidate for starting the new orders)
    matrix_vector = matrix.reshape(-1)

    max_atoms = (n_size-1)*2
    if n_add > max_atoms:
        raise Exception("Too many atoms are being added on the surface; Reduce no_added_atoms value to be less than",max_atoms)
    for i in np.arange(n_add):
        try:
            num  = np.random.choice(matrix_vector)
        except ValueError:
            print("There is no more space on the surface to place new sites")
            break

        else:
            sc = n_size*2 #Scale of which to scale locations as they are added to the diamond (8 for 4x4)
            new_site_left = [num,num+1,num+2] #Creating Left Side of Diamond
            new_site_right = [(x + sc -1) for x in new_site_left] #Creating right side of diamond


            new_site_nos = new_site_left + new_site_right #Concatenating

            new_site_type = [hcp, fcc] * len(new_site_left) #Creating list of corresponding site types

            #using the new site_no and site_type arrays to change the dataframe
            for i in np.arange(len(new_site_nos)):
                new_df.iloc[new_site_nos[i]-1,3] = new_site_type[i]

            #removing the chosen new sites from list of options
            matrix_vector = np.array([i for i in matrix_vector if i not in new_site_nos])

    #             #ADDING EXTREME EXCLUSION
    #             proximity_base = [num+sc, num-sc]
    #             proximity_top = [x + 2 for x in proximity_base]
    #             proximity_bottom = [x - 2 for x in proximity_base]
    #             proximity = proximity_base + proximity_top + proximity_bottom
    #             #removing all proximity sites as potential new sites
    #             matrix_vector = np.array([i for i in matrix_vector if i not in proximity])

    return new_df

In [5]:
new_site_nos = [1212,1213,1214,1283,1284,1285]
np.arange(len(new_site_nos))

array([0, 1, 2, 3, 4, 5])

In [6]:
# new = sequence_hex(o,1,3,4)

In [7]:
# new

In [8]:
no_additional_sites = len(additional_sites)
total_all_site_type = def_no_site_types+no_additional_sites
list_of_all_site_number_strings = [str(x) for x in np.arange(1, total_all_site_type+1)]

file_name = Output_File_name
ofile=open(file_name,'w')
ofile.write("lattice explicit\n")
ofile.write("\n")
ofile.write("n_sites         "+str(Tot_n_sites)+"\n")
ofile.write("max_coord       "+str(max_cord)+"\n")
ofile.write("n_site_types    "+str(total_all_site_type)+"\n")
ofile.write("site_type_names "+' '.join(site_names)+' '+' '.join(additional_sites) +"\n")
ofile.write("\n")
ofile.write("##site_types    "+' '.join(list_of_all_site_number_strings) +"\n")
ofile.write("\n")
ofile.write("lattice_structure\n")

# Making new lattice structure dataframe with replacement values
new = sequence_hex(o,no_added_atoms,added_site_type_number[1],added_site_type_number[0])

#Writing each line of the lattice structure accordingly
for i in np.arange(np.shape(new)[0]):
    ofile.write("         %s"%(str(new.iloc[i,0]).rjust(5)) +"       %s"%(str(new.iloc[i,1]).rjust(5))
                +"       %s"%(str(new.iloc[i,2]).rjust(5))
                +"       %s"%(str(new.iloc[i,3]).rjust(5)))
    
    #Adding the neighbors:
    for j in new.iloc[i,4:]:
        ofile.write("     %s"%(str(j)).rjust(5))
        
    ofile.write("\n")
ofile.close()